In [2]:
pip install flask pandas geopandas matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [flask]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

# ================================
# 1. Charger les 2 CSV bruts
# ================================

# ⚠️ Remplace par les bons noms si nécessaire
df_densite = pd.read_csv("../data/Densite.csv")
df_effectif = pd.read_csv("../data/Effectifs.csv")

print("Colonnes densité :", df_densite.columns.tolist())
print("Colonnes effectif :", df_effectif.columns.tolist())

Colonnes densité : ['dept_code', '01-Médecine générale', '02-Chirurgie', '03-Ophtalmologie', '04-Oto-rhino-laryngologie', '05-Anesthésie-Réanimation', '06-Biologie médicale', '07-Cardiologie', '08-Dermatologie', '09-Gastro-entérologie', '10-Gynécologie', '11-Pédiatrie', '12-Pneumologie', '13-Psychiatrie et neuropsychiatrie', '14-Plateaux techniques', '15-Autre spécialité', 'Total général']
Colonnes effectif : ['dept_code', '01-Médecine générale', '02-Chirurgie', '03-Ophtalmologie', '04-Oto-rhino-laryngologie', '05-Anesthésie-Réanimation', '06-Biologie médicale', '07-Cardiologie', '08-Dermatologie', '09-Gastro-entérologie', '10-Gynécologie', '11-Pédiatrie', '12-Pneumologie', '13-Psychiatrie et neuropsychiatrie', '14-Plateaux techniques', '15-Autre spécialité', 'Total général']


In [8]:
specialite_cols = [
    col for col in df_densite.columns
    if col not in ["dept_code", "Total général"]
]

print("Colonnes spécialités détectées :", specialite_cols)

Colonnes spécialités détectées : ['01-Médecine générale', '02-Chirurgie', '03-Ophtalmologie', '04-Oto-rhino-laryngologie', '05-Anesthésie-Réanimation', '06-Biologie médicale', '07-Cardiologie', '08-Dermatologie', '09-Gastro-entérologie', '10-Gynécologie', '11-Pédiatrie', '12-Pneumologie', '13-Psychiatrie et neuropsychiatrie', '14-Plateaux techniques', '15-Autre spécialité']


In [9]:
# Densité
densite_long = df_densite.melt(
    id_vars="dept_code",
    value_vars=specialite_cols,
    var_name="specialite",
    value_name="densite"
)

# Effectif
effectif_long = df_effectif.melt(
    id_vars="dept_code",
    value_vars=specialite_cols,
    var_name="specialite",
    value_name="effectif"
)

# Fusion densité + effectif
df_long = densite_long.merge(
    effectif_long,
    on=["dept_code", "specialite"],
    how="left"
)

In [10]:
def clean_dept(code):
    s = str(code).strip()
    if s in ["2A", "2B"]:
        return s
    if s.isdigit():
        return s.zfill(2)  # 1 → 01, 2 → 02, 9 → 09
    return s

df_long["dept_code_clean"] = df_long["dept_code"].apply(clean_dept)


In [11]:
output_file = "rpps_long_clean.csv"
df_long.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"\nFichier généré avec succès : {output_file}")
print(df_long.head())


Fichier généré avec succès : rpps_long_clean.csv
  dept_code            specialite densite  effectif dept_code_clean
0         1  01-Médecine générale  100,11       672              01
1         2  01-Médecine générale  101,23       532              02
2         3  01-Médecine générale  131,75       441              03
3         4  01-Médecine générale  165,09       276              04
4         5  01-Médecine générale  273,86       388              05


In [13]:
print(df_densite.columns)


Index(['dept_code', '01-Médecine générale', '02-Chirurgie', '03-Ophtalmologie',
       '04-Oto-rhino-laryngologie', '05-Anesthésie-Réanimation',
       '06-Biologie médicale', '07-Cardiologie', '08-Dermatologie',
       '09-Gastro-entérologie', '10-Gynécologie', '11-Pédiatrie',
       '12-Pneumologie', '13-Psychiatrie et neuropsychiatrie',
       '14-Plateaux techniques', '15-Autre spécialité', 'Total général'],
      dtype='object')


In [14]:
print(df_effectif.columns)

Index(['dept_code', '01-Médecine générale', '02-Chirurgie', '03-Ophtalmologie',
       '04-Oto-rhino-laryngologie', '05-Anesthésie-Réanimation',
       '06-Biologie médicale', '07-Cardiologie', '08-Dermatologie',
       '09-Gastro-entérologie', '10-Gynécologie', '11-Pédiatrie',
       '12-Pneumologie', '13-Psychiatrie et neuropsychiatrie',
       '14-Plateaux techniques', '15-Autre spécialité', 'Total général'],
      dtype='object')


In [18]:
from flask import Flask, render_template, request, send_file
import pandas as pd
import geopandas as gpd
import matplotlib
matplotlib.use("Agg")  # backend sans interface graphique
import matplotlib.pyplot as plt
import io

app = Flask(__name__)

# === Charger les données au démarrage de l'appli ===

# 1. Données RPPS (long format)
df = pd.read_csv("rpps_long_clean.csv")

# 2. Contours des départements
gdf_dept = gpd.read_file("../data/departements.geojson")

# On suppose que le code département est dans la colonne 'code'
# Si c'est 'code_insee' ou autre, adapte ici :
gdf_dept = gdf_dept.rename(columns={"code": "dept_code_clean"})

# 3. Liste des spécialités pour le menu déroulant
specialites = (
    df[["specialite", "specialite"]]
    .drop_duplicates()
    .sort_values("specialite")
    .to_dict(orient="records")
)


@app.route("/")
def index():
    # spécialité sélectionnée (sinon par défaut la première)
    spec = request.args.get("specialite")
    if spec is None and specialites:
        spec = specialites[0]["specialite"]

    return render_template(
        "index.html",
        specialites=specialites,
        selected_specialite=spec,
    )


@app.route("/map.png")
def map_png():
    spec = request.args.get("specialite")
    if spec is None and specialites:
        spec = specialites[0]["specialite"]

    # Filtrer les données pour la spécialité choisie
    df_spec = df[df["specialite"] == spec].copy()

    # Fusion avec les départements
    gdf_merged = gdf_dept.merge(
        df_spec,
        on="dept_code_clean",
        how="left"
    )

    # Choisir la métrique à afficher : 'densite' ou 'effectif'
    metric = request.args.get("metric", "effectif")
    if metric not in ["densite", "effectif"]:
        metric = "effectif"

    # Créer la figure matplotlib
    fig, ax = plt.subplots(figsize=(7, 7))

    gdf_merged.plot(
        column=metric,
        ax=ax,
        legend=True,
        missing_kwds={
            "color": "lightgrey",
            "label": "Pas de données"
        }
    )

    # Titre sympa
    label = df_spec["specialite"].iloc[0] if not df_spec.empty else spec
    ax.set_title(f"{metric.capitalize()} - {label}")
    ax.axis("off")
    plt.tight_layout()

    # Sauvegarder dans un buffer mémoire au format PNG
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=120)
    plt.close(fig)
    buf.seek(0)

    return send_file(buf, mimetype="image/png")


if __name__ == "__main__":
    app.run(debug=True)


ValueError: The column label 'specialite' is not unique.

In [17]:
print(df.columns.tolist())

['dept_code', 'specialite', 'densite', 'effectif', 'dept_code_clean']
